<a href="https://colab.research.google.com/github/sandoval19/Santander-challange/blob/master/Santander_CPT_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("!Hola")

!Hola


In [2]:
print("por fin")

por fin
